# Using from X Import Y in Python

In [ ]:
from X import Y

In [ ]:
import foobar

In [ ]:
from foobar import hello

In [ ]:
from foobar import hello, x

Note that I keep saying, "the current namespace." That's because "import", like "def", is a way to define a variable. When you use "def", you're both creating a function object and setting a variable (the function name) to point to that function object.  And when you use "import", you're both creating a module object, and setting a variable (the module name) to point to that module object.

But all variables can be global or local -- and modules are no different.  For example:

In [3]:
os

NameError: name 'os' is not defined

In [4]:
def foo():
    import os
    print(os.sep)
foo()

/


In [5]:
os

NameError: name 'os' is not defined

In the above code, we start off with "os" not being defined as a variable.  In the function "foo", we import the module "os", giving us access to its contents. However, the variable "os" is local to the function "foo", meaning that when the function returns, the variable is no longer available.

I should note that while you can use an "import" statement anywhere, it's pretty rare in my experience to have it anywhere but at the global scope.

In the case of "from-import", the imported variables are defined inside of the current namespace.  Once again, it's pretty rare to use it anywhere outside of the global scope.

One oddity of "from-import" is that the module itself isn't defined. For example:

In [6]:
from os import sep
sep

'/'

In [7]:
or

SyntaxError: invalid syntax (<ipython-input-7-b8dec49073b8>, line 1)

You are interested in importing specific names from the module, rather than the entire module.

This raises the question of what happens when you execute "from-import". Is the entire module loaded?  Or does Python somehow just pick through the module file, grabbing only the values that are of interest?

We can check this; here is the same simple module that we defined last week:

In [8]:
from mymod import x, hello

Hello from mymod.py!


then what does Python do? The answer is that, as with "import", it executes the entire file, top to bottom. The only difference is that the variables named in the "from-import" statement are then aliased in the current namespace, and the module's name is not defined.

The module is still available in sys.modules, which means that the second time that you import it, the "print" statement is no longer executed:

In [9]:
from mymod import hello

In [10]:
x

100

In [11]:
mymod.x

NameError: name 'mymod' is not defined

In [12]:
import sys
sys.modules['mymod'].x

100

In [14]:
from mymod import x
x

100

In the above code, we first load the "hello" functino from "mymod".  When "from-import" is executing, "print" in the module executes. We then see that "x", which is also defined in mymod, hasn't been defined in the local namespace.

However, was "x" defined in the module? We can use sys.modules, which stores all of the modules imported into Python, to answer this question. And as we see above, it was definitely defined.

So: "from-import" loads the entire module, and puts the module in sys.modules. It then creates aliases to the specified names in the local namespace.

And if you're using "from-import" because you want to save memory, or don't want to load an entire module, that's obviously bad news.

Now, many people use the following syntax:
    `from mymod import *`
I want to say this very clearly: Don't do this in your production code.  If it helps you while you're development, then power to you, and feel free. But doing this in production code is asking for trouble.

The reason is that "from-import", as we've seen, defines variables in the local namespace.  When you say "from import *", you're saying that it would be totally OK for the module's variables to overwrite the variables that you have defined in the current namespace. In other words, if I say
    `from mymod import *`
and I previously had a variable named "x", then the new value of "x" will be based on the module's value.  Imagine upgrading to a new version of the module, and not noticing that it has an "x" variable.  Oops!

I thus strongly suggest that you avoid using "from-import-*".  That said, it has a few interesting properties:

For starters, "from-import" ignores names that start with an underscore (_) character.  Thus, if mymod.py is: